In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import nglview as nv
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
from IPython.display import display, display_markdown
from ipywidgets import Layout
from pathlib import Path

import parmed as pmd
import re

import hublib.use
from hublib.ui import FileUpload, Download
from hublib.cmd import runCommand

from scipy.ndimage import gaussian_filter

%matplotlib notebook
%use gromacs-5.1.4
%use pymol-1.8.4

np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)

<IPython.core.display.Javascript object>

In [4]:
class pigment_class:
    def __init__(self, name, BaseNames, OptNames):
        self.name = name
        self.BaseNames = BaseNames
        self.OptNames = OptNames
        
# XNames is the list of atoms names that should be used to identify the pigment type 
# within a given class. 
class pigment_type:
    def __init__(self, ResName, StdName, pClass, XAtNames, NSDNames, TrESPNames, Q10, diplength):
        self.pclass = pClass,
        self.name = ResName
        self.stdname = StdName
        self.xnames = XAtNames
        self.nnames = NSDNames
        self.tnames = TrESPNames
        self.q10 = Q10
        self.diplength = diplength
        
class pigment:
    def __init__(self, idx, ptype, alist, resname, chain, resnum, atnames, atcoords):
        self.idx = idx
        self.ptype = ptype
        self.alist = alist
        self.resname = resname
        self.chain = chain
        self.resnum = resnum
        self.atnames = atnames
        self.atcoords = atcoords
        self.widget = []

# Returns a list of all residue indices (in struc)
# that contain all required atoms
def find_pigments(NameList, instruc):

    SelStr = "(:0-"+str(len(instruc.residues)) + ")"
    for ndx in range(1, len(NameList)):
        tstruc = instruc.view[SelStr + ' & (@'+NameList[ndx] + ")"]
        SelNdx = []
        for at in tstruc:
            SelNdx.append(at.residue.idx)
        SelStr = "(:" + ",".join([str(i+1) for i in SelNdx]) + ")"
        if len(SelNdx)==0:
            break

    return SelNdx
    
# Now check which pigment types can be *excluded* based on the atoms
# that are present in each pigment. NB: We do *not* require all atoms
# to be present. 
# The return argument AList is a list of pigment types that *are* allowed. 
def eliminate_types(PigNdcs, PClass, TypeList, UNK, instruc):
    
    AList = []
    for ndx in PigNdcs:
        tstruc = instruc[':'+str(ndx+1)]
        
        # For each atom in the pigment, check whether it's allowed
        # under each pigment type. 
        TypeAllowed = [1 for i in range(0,len(TypeList))]
        for at in tstruc:
            # First check if it's a base atom or optional atom. If not, we check 
            # the characteristic atoms for each pigment. 
            if (PClass.BaseNames.count(at.name)==0) and (PClass.OptNames.count(at.name)==0):
                for tndx in range(0, len(TypeList)):
                    if TypeList[tndx].xnames.count(at.name)==0:
                        TypeAllowed[tndx] = 0
        
        alist = []
        for tndx in range(0, len(TypeList)):
            if TypeAllowed[tndx]:
                alist.append(TypeList[tndx])
                
        if len(alist)==0:
            alist = [UNK]
        
        AList.append(alist)
        
    return AList


# MList[p] is a list of matched types for pigment p
def match_types(PigNdcs, AList, instruc):
    
    MList = []
    # Loop through pigments
    for p in range(0, len(PigNdcs)):
        ndx = PigNdcs[p]
        
        tstruc = instruc[':'+str(ndx+1)]
        
        # alist is a list of pigment types allowed for pigment p
        alist = AList[p]
        
        # mlist will store a list of types that match the pigment exactly
        mlist = []
        
        # Loop through possible types
        for t in range(0, len(alist)):
            
            # Start assuming the pigment matches the type
            match = True
            typ = alist[t]
            for name in typ.xnames:
                
                # If any atom is not found, set match to False and break
                if len(tstruc['@'+name].atoms)==0:
                    match = False
                    break
                
            # If we have a match, append it to the pigment's match list
            if match:
                mlist.append(typ)
                
        MList.append(mlist)
    return MList

# 1. Assign all pigments to the top entry in MList[p] if non-empty
# 2. For each pigment still unassigned, check whether any pigments with the 
#    same name have already been assigned. If so, assign to the same type. 
# 3. For each pigment still unassigned, assign to the top entry in AList[p] if non-empty
def assign_pigments(PigNdcs, MList, AList, instruc):
    TList = []
    
    # 1. Assign by MList
    for p in range(0, len(PigNdcs)):
        if len(MList[p])>0:
            TList.append(MList[p][0])
        else:
            TList.append([])
            
    # 2. Assign by name
    for p in range(0, len(PigNdcs)):
        if TList[p]==[]:
            rname = struc.residues[PigNdcs[p]].name
            
            # Check if any assigned pigment has a matching name 
            for pref in range(0, len(PigNdcs)):
                if (TList[pref]!=[]) and (rname==struc.residues[PigNdcs[pref]].name):
                    TList[p] = TList[pref]
                    break
                    
    # 3. If pigments are still unmatched, assign to top alist entry
    for p in range(0, len(PigNdcs)):
        if TList[p]==[]:
            TList[p] = AList[p][0]
                
    return TList

In [5]:
# These names are considered essential to all Chl/BChl molecules
# Without these, we can do no calculations. 
ChlBaseNames = []
for ring in ['A', 'B', 'C', 'D']:
    for at in ['N', 'C1', 'C2', 'C3', 'C4', 'CH']:
        ChlBaseNames.append(at+ring)
ChlBaseNames.append('CAD')
ChlBaseNames.append('CBD')

# These are the atoms of the phytol tail
PhytolNames = []
for n in range(1, 21):
    PhytolNames.append('C'+str(n))
    
# All pigments in the class have these atoms, but we can identify the class without them
ChlOptNames = ['CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'CMA', 'CMB', 'CAB', 'CMC', 'CAC', 'CBC', 'CMD', 'OBD'] + PhytolNames


# The tetpy class describes common tetrapyrrole pigments
tetpy = pigment_class('tetrapyrrole', ChlBaseNames, ChlOptNames)


# These atoms can be used to distinguish between different pigment types
# Note: 
#    The heavy atoms of Chl a, Chl c1, Chl c2, and BChl g are identical (excluding the phytol tail)
#    The heavy atoms of BChl a and BChl b are identical
CLANames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED']
CLBNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED', 'OMC']
CLDNames = ['MG',        'CGD', 'O1D', 'O2D', 'CED',        'OBB']
CLFNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED',               'OMB'] 
BCANames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED',        'OBB']
BCBNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED',        'OBB']
BCCNames = ['MG', 'CBB',                                    'OBB',        'CIB']
BCDNames = ['MG', 'CBB',                                    'OBB',               'CND']
BCENames = ['MG', 'CBB',                             'OMC', 'OBB',        'CIB', 'CND']
BCFNames = ['MG', 'CBB',                             'OMC', 'OBB',               'CND']
BCGNames = ['MG', 'CBB', 'CGD', 'O1D', 'O2D', 'CED']

# The corresponding Pheo names are the same, excluding the first entry (MG)
PHANames = CLANames[1:]
PHBNames = CLBNames[1:]
BPANames = BCANames[1:]


CLA = pigment_type(
    'Chl a',
    'CLA',
    tetpy,
    CLANames,
    [],
    ['MG', 'CHA', 'CHB', 'CHC', 'CHD', 'NA', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'NB', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'CBB', 'NC', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'CAC', 'CBC', 'ND', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'OBD', 'CBD', 'CGD', 'O1D', 'O2D', 'CED', 'C1'],
    (1e-3)*(4.80326e-10)*np.array([-21.674,106.779,-48.696,-98.725,72.726,31.683,-130.820,10.048,2.389,77.983,5.556,-1.050,0.735,-8.040,-1.330,7.091,-62.297,81.122,4.777,-9.204,106.271,16.963,10.681,34.947,-12.166,83.646,-7.420,-1.126,-44.008,-5.161,8.020,1.012,108.292,-110.812,-11.981,8.799,-125.044,-25.156,-19.250,-20.044,-11.238,5.979,-5.388,1.811,-5.256,-1.424]),
    4.2e-18
)

CLB = pigment_type(
    'Chl b',
    'CLB',
    tetpy,
    CLBNames, 
    [],
    ['MG', 'CHA', 'CHB', 'CHC', 'CHD', 'NA', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'NB', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'CBB', 'NC', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'OMC', 'CAC', 'CBC', 'ND', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'OBD', 'CBD', 'CGD', 'O1D', 'O2D', 'CED'],
    (1e-3)*(4.80326e-10)*np.array([-15.512,75.628,-57.844,-107.433,86.081,11.573,-99.670,7.454,7.749,77.389,3.351,-2.720,-5.754,4.859,0.650,-3.916,-73.990,85.402,-2.140,-5.618,108.571,14.838,1.978,24.357,-21.650,79.538,5.163,-5.803,-31.957,-13.413,-1.389,6.975,2.317,90.881,-88.170,-13.166,15.953,-98.277,-16.435,-28.234,-14.273,-0.035,7.743,-7.166,0.641,-4.526]),
    3.60e-18
)

CLD = pigment_type(
    'Chl d',
    'CLD',
    tetpy,
    CLDNames, 
    [],
    [],
    np.array([]),
    0.0
)

CLF = pigment_type(
    'Chl f',
    'CLF',
    tetpy,
    CLFNames,
    [],
    [],
    np.array([]),
    0.0
)

BCA = pigment_type(
    'BChl a',
    'BCA',
    tetpy,
    BCANames, 
    [],
    ['MG', 'CHA', 'CHB', 'CHC', 'CHD', 'NA', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'NB', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'OBB', 'CBB', 'NC', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'CAC', 'CBC', 'ND', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'OBD', 'CBD', 'CGD', 'O1D', 'O2D', 'CED', 'C1'],
    (1e-3)*(4.80326e-10)*np.array([-42.217,141.478,-14.988,-45.427,41.071,65.489,-134.776,19.419,2.723,17.043,11.656,-15.907,9.451,-5.063,5.421,-3.656,-28.216,87.663,-0.905,67.447,33.680,32.745,-25.091,37.208,19.037,16.855,80.984,-19.196,28.047,-116.550,7.787,0.863,-9.640,135.026,-111.514,-22.400,36.878,-200.720,-31.393,-35.896,-20.922,-2.346,-2.023,-8.885,7.086,-7.868,0.543]),
    5.477e-18
)

BCB = pigment_type(
    'BChl b',
    'BCB',
    tetpy,
    BCBNames, 
    [],
    [],
    np.array([]),
    0.0
)

BCC = pigment_type(
    'BChl c',
    'BCC',
    tetpy,
    BCCNames,
    [],
    [],
    np.array([]),
    0.0
)

BCD = pigment_type(
    'BChl d',
    'BCD',
    tetpy,
    BCDNames, 
    [],
    [],
    np.array([]),
    0.0
)

BCE = pigment_type(
    'BChl e',
    'BCE',
    tetpy,
    BCENames, 
    [],
    [],
    np.array([]),
    0.0
)

BCF = pigment_type(
    'BChl f',
    'BCF',
    tetpy,
    BCFNames, 
    [],
    [],
    np.array([]),
    0.0
)

BCG = pigment_type(
    'BChl g',
    'BCG',
    tetpy,
    BCGNames, 
    [],
    [],
    np.array([]),
    0.0
)

PHA = pigment_type(
    'Pheo a',
    'PHA',
    tetpy,
    PHANames,
    [],
    ['CHA', 'CHB', 'CHC', 'CHD', 'NA', 'C1A', 'C2A', 'C3A', 'C4A', 'CMA', 'CAA', 'CBA', 'CGA', 'O1A', 'O2A', 'NB', 'C1B', 'C2B', 'C3B', 'C4B', 'CMB', 'CAB', 'CBB', 'NC', 'C1C', 'C2C', 'C3C', 'C4C', 'CMC', 'CAC', 'CBC', 'ND', 'C1D', 'C2D', 'C3D', 'C4D', 'CMD', 'CAD', 'OBD', 'CBD', 'CGD', 'O1D', 'O2D', 'CED'],
    (1e-3)*(4.80326e-10)*np.array([76.834,-38.800,-89.458,62.844,47.290,-86.246,7.560,-1.965,70.215,4.115,-2.753,1.651,-3.830,-1.585,2.947,-62.801,66.190,8.864,-9.269,90.283,15.078,9.556,28.399,-25.439,73.639,9.381,-20.445,-24.827,-3.833,6.785,-0.061,87.071,-87.246,-8.812,2.558,-94.919,-9.925,-16.035,-18.630,-8.844,4.351,-4.024,2.224,-5.226]),
    3.50e-18
)

UNK = pigment_type(
    'Unknown',
    'UNK',
    tetpy,
    [],
    [],
    [],
    np.array([]),
    0.0
)


TetPyList = [
    CLA, CLB, #CLD, CLF, 
    BCA, #BCB, BCC, BCD, BCE, BCF, BCG,
    PHA, 
]

In [6]:


# Global variables:

# Main ParmEd structure
struc = pmd.structure.Structure()

# Representation list for main structure
mainreps = list()

# Representation list for dipoles
dipreps = list()

# List of chains in struc
ChainList = []

# List of identified pigments
PigList = []

##################################################################
# Main frame layout:
##################################################################

# mainbox is the top widget. It contains:
#  pdbview -- an NGLWidget used to display the loaded structures
#  mainacc -- an accordion widget containing the following VBoxes:
#    strucbox -- contains the "Load" interface
#    selbox -- contains the "Select" interface
#    writebox -- contains the "Write PDB" interface
#    excbox -- contains the "Prepare Exciton Model" interface
#    mdbox -- contains the "Prepare MD Model" interface

# Structure viewer:
pdbview = nv.NGLWidget()
pdbview._set_size('500px', '500px')


##################################################################
# strucbox: Interface for loading molecular structures
##################################################################

# pdbid: Text entry box for PDB ID 
# pdbidlbl: Label for pdbid
# pdbfetch: Button to fetch PDB from the RCSB databank
# pdbup: Button to upload PDB file
# pdbuplbl: Label for pdbup


pdbid = widgets.Text(
    value='2DRE',
    placeholder='Type something',
    layout = widgets.Layout(width='1.5cm'),
    disabled=False
)

pdbidlbl = widgets.Label(value='Enter PDB ID:', width='4cm')

pdbfetch = widgets.Button(
    description='Fetch',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to display the pdb file',
    layout = widgets.Layout(width='2.25cm'),
    icon='' # (FontAwesome names without the `fa-` prefix)
)

pdbup = widgets.FileUpload(
    accept='.pdb',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

pdbuplbl = widgets.Label(value='Or upload file:', width='4cm')

strucbox = widgets.VBox([widgets.HBox([pdbidlbl, pdbid, pdbfetch]), widgets.HBox([pdbuplbl, pdbup])])

# Executed with pdbfetch is clicked
def pdbfetch_onclick(b):

    url = 'http://files.rcsb.org/download/' + pdbid.value + '.pdb'
    r = requests.get(url, allow_redirects=True)
    if(r.status_code!=200):
        print('Invalid PDB code. Please try again.')
    else:
        fname = 'pdb/'+pdbid.value+'.pdb'
        wfd = open(fname, 'wb')
        wfd.write(r.content)
        wfd.close()
        init_struc(fname)

# Executed when pdbup is clicked
def pdbup_on_value_change(change):
    for item in pdbup.value:
        fname = item
    
    with open("pdb/"+fname, "wb") as fp:
        fp.write(pdbup.value[fname]["content"])
    fp.close()
    init_struc("pdb/"+fname)
    
pdbup.observe(pdbup_on_value_change, 'value')

def build_dipstruc(struc):
    
    dipstruc = pmd.structure.Structure()
    
    dipcoords = []
    porphlist = list()
    porphtxt = '('
    PorphAts = ['NA', 'NB', 'NC', 'ND']
    for res in struc.residues:

        # Check if it's a porphyrrin
        foundNs = np.zeros((len(PorphAts),))
        patnums = np.zeros((len(PorphAts),), dtype='int')
        for at in res:
            for n in range(0, len(PorphAts)):
                if at.name==PorphAts[n]:
                    foundNs[n] = 1
                    patnums[n] = at.idx

        # If we located all four ring N atoms
        if np.sum(foundNs)==4:
            cenvec = 0.5*(struc.coordinates[patnums[1]] + struc.coordinates[patnums[3]])
            dipvec = (struc.coordinates[patnums[3]] - struc.coordinates[patnums[1]])
            vStart = cenvec - 1.25*dipvec
            vStop = cenvec + 1.25*dipvec
            Nats = 10
            for a in range(0, Nats+1):
                xyz = (float(a)/float(Nats))*vStart + (1 - float(a)/float(Nats))*vStop
                dipstruc.add_atom(pmd.topologyobjects.Atom(name='N'), 'Dip', res.idx, chain=res.chain)
                dipcoords.append(xyz)

            porphlist.append(res.idx)
            if len(porphtxt)>1:
                porphtxt += ' OR '
            porphtxt += str(res.idx+1)
    porphtxt += ')'

    # If any porphyrrins have been located, add the coordinates to the dipole list. 
    if len(dipcoords)>0:
        dipstruc.coordinates = np.array(dipcoords)
    else: 
        porphtxt = ""
        
    return dipstruc, porphtxt
    

# init_struc() loads a structure from the provided file name
# and initializes the structure view representations and chainlists
#
# Outside references:
#   sets pigbox.children and chainbox.children each time a new structure is loaded
#
# Relies on environment variables:
#   UNK
#   tetpy
#   TetPyList
def init_struc(fname):

    global mainreps
    global dipreps
    global struc
    global ChainList
    global chainbox
    global writebt
    global pigbox
    global PigList
    
    speclbl_out.clear_output()
    
    # Clear pdbview stage
    while len(pdbview._ngl_component_ids)>0:
        pdbview.remove_component(pdbview._ngl_component_ids[0])
    
    # Reset rep and chain lists
    mainreps = list()
    dipreps = list()
    
    struc = pmd.load_file(fname)
    dipstruc, porphtxt = build_dipstruc(struc)
    
    ChainList = []
    for res in struc.residues:
        # If the chain is not already listed, add it
        if ChainList.count(res.chain)==0:
            ChainList.append(res.chain)
    
    chaintxt = '('
    for chain in ChainList:
        if len(chaintxt)>1:
            chaintxt += ' OR '
        chaintxt += ':' + chain
    chaintxt += ')'

    pdbview.add_trajectory(struc)
    mainreps = list()
    mainreps.append({"type": "cartoon", "params": {"color": "grey", "sele": "(protein) AND " + chaintxt, "opacity": "0.2"}})
    if len(porphtxt)>0:
        mainreps.append({"type": "licorice", "params": {"color": "green", "sele": porphtxt + ' AND ' + chaintxt, "opacity": "1.0"}})
    pdbview.set_representations(mainreps, component=0)

    if len(dipstruc.atoms)>0:
        pdbview.add_trajectory(dipstruc)
        dipreps = [{"type": "licorice", "params": {"color": "red", "sele": chaintxt, "opacity": "1", "radius": "0.35"}}]
        pdbview.set_representations(dipreps, component=1)

    # Assign tetrapyrrole types:
    # 1. Identify tetrapyrrole rings
    PigNdcs = find_pigments(tetpy.BaseNames, struc)

    # 2. Check which types are definitely excluded for each pigment
    alist = eliminate_types(PigNdcs, tetpy, TetPyList, UNK, struc)

    # 3. Now check if all xatoms of each type are present
    mlist = match_types(PigNdcs, alist, struc)
    
    # 4. Based on this data, assign pigment types
    tlist = assign_pigments(PigNdcs, mlist, alist, struc)
    
    PigList = []
    # Record pigments:
    for p in range(0, len(PigNdcs)):
        ndx = PigNdcs[p]
        res = struc.residues[ndx]
        atom_ndcs = []
        atom_names = []
        for at in res:
            atom_ndcs.append(at.idx)
            atom_names.append(at.name)
        atom_ndcs = np.array(atom_ndcs, dtype='int')
        atcoords = struc.coordinates[atom_ndcs]
        PigList.append(pigment(ndx, tlist[p], alist[p], res.name, res.chain, res.number, atom_names, atcoords))
    
    build_pigbox()
    
    chaincbs = []
    for chain in ChainList:
        chaincbs.append(widgets.Checkbox(value=True, description=chain,indent=False))
    for cb in chaincbs:
        cb.observe(update_chains)
    chainbox.children = chaincbs
    writebt.disabled = False
    
    excgo.disabled = False
    coupselect.disabled = False
    siteselect.disabled = False

# Syncs structure display to selected chains in chainbox
def update_chains(b):
    global dipreps
    global mainreps 
    global ChainList
    
    ChainList = []
    for cb in chainbox.children:
        if cb.value==True:
            ChainList.append(cb.description)
        
    chaintxt = ''
    for chain in ChainList:
        if len(chaintxt)>0:
            chaintxt += " OR "
        chaintxt += ":" + chain
        
    for rep in mainreps:
        #rep['params']['sele'] = re.sub('(:[^)]+)', chaintxt, rep['params']['sele'])
        splt = rep['params']['sele'].split('(')
        newstr = ''
        for n in range(1, len(splt)-1):
            newstr += "(" + splt[n]
        newstr += '(' + chaintxt + ")"
        rep['params']['sele'] = newstr
    pdbview.set_representations(mainreps, component=0)
    
    #dipreps[0]['params']['sele'] = re.sub('(:[^)]+)', chaintxt, rep['params']['sele'])
    if len(dipreps)>0:
        splt = dipreps[0]['params']['sele'].split('(')
        newstr = ''
        for n in range(1, len(splt)-1):
            newstr += "(" + splt[n]
        newstr += '(' + chaintxt + ")"
        dipreps[0]['params']['sele'] = newstr
        pdbview.set_representations(dipreps, component=1)
    
pdbfetch.on_click(pdbfetch_onclick)

##################################################################
# selbox: Interface for selecting chains and residues
##################################################################

# Label for chain list
chainlbl = widgets.Label(value='Chain:')

# The children (i.e., options) of chainbox are set in pdbfetch_onclick()
# when a new structure is loaded. 
chainbox = widgets.VBox([])

selbox = widgets.VBox([chainlbl, chainbox])


##################################################################
# writebox: Interface for writing PDB output
##################################################################

writetxt = widgets.Text(value='test.pdb', description='File Name:', disabled=False, layout=widgets.Layout(width='5cm'))
writebt = widgets.Button(description='Write',tooltip='Click to write PDB file',layout=widgets.Layout(width='1.5cm'), disabled=True)

def pdbwrite_onclick(b):
    fname = "pdb/" + writetxt.value
    
    # First identify which chains should be written
    # Loop through chain-selection check-boxes and
    # add a parmed structure object for each chain
    strucList = []
    for cb in chainbox.children:
        if cb.value==True:
            strucList.append(struc[cb.description,:,:])
    
    # Now combine all sub-structures into a single structure
    # for writing to pdb. 
    selstruc = []
    if len(strucList)>0:
        selstruc = strucList[0]
        for n in range(1, len(strucList)):
            selstruc = selstruc + strucList[n]
        selstruc.write_pdb(fname)
        disptext = 'Structure successfully written to file <a href=\"' + fname + '\" target="_blank">'+fname+'</a>'
        display_markdown(disptext, raw=True)
        
writebt.on_click(pdbwrite_onclick)

writebox = widgets.HBox([writetxt, writebt])


##################################################################
# excbox: Interface for building exciton models
##################################################################

def calculate_tresp():
    
    global PigList
    global ChainList
    
    # First create list of chain-selected pigments
    SelPigs = []
    for pig in PigList:
        if ChainList.count(pig.chain)>0:
            SelPigs.append(pig)
    Npigs = len(SelPigs)

    Npigs = len(SelPigs)
    CoupMat = np.zeros((Npigs,Npigs))
    Dips = []
    h = 6.62607015e-34 #J*s
    c = 2.998e10 #cm/s
    
    Erg2J = 1.0e-7
    nm2cm = 1.0e-8
    
    # Now check whether TrESP atoms are available for *all*
    # selected pigments. 
    missing_atoms = False
    
    # Loop over selected atoms
    for p in range(0, Npigs):
        pig = SelPigs[p]
        # Loop over TrESP atoms
        for name in pig.ptype.tnames:
            if pig.atnames.count(name)==0:
                print('Error: Could not locate TrESP atom ' + name + " in pigment " + pig.resname + " " + pig.chain + " " + str(pig.resnum))
                print('Aborting TrESP calculation. Please try point-dipole approximation (PDA) or add missing atoms.')
                missing_atoms = True
                break
            if missing_atoms:
                break
                
    if missing_atoms==False:
        # Build list of TrESP coordinates for each pigment
        # NB: Units are converted to cm!
        CoordList = []
        for p in range(0, Npigs):
            pig = SelPigs[p]
            coords = []
            for name in pig.ptype.tnames:
                ndx = pig.atnames.index(name)
                coords.append(pig.atcoords[ndx])
            CoordList.append(np.array(coords)*nm2cm)
            
        # Calculate pigment dipole lengths (as per default TrESP charges)
        # We'll divide charges for each pigment by the ratio of the *calculated*
        # dipole length to the *standard* dipole length. 
        # Charges are in units of cm^3/2 g^1/2 s^−1.
        DipLengths = []
        for p in range(0, Npigs):
            pig = SelPigs[p]
            dip = np.zeros((3,))
            for n in range(0, len(pig.ptype.tnames)):
                dip += pig.ptype.q10[n]*CoordList[p][n]
                
            DipLengths.append(np.linalg.norm(dip))
            Dips.append(dip/np.linalg.norm(dip))
        Dips = np.array(Dips)
            
        # Calculate interactions
        for p1 in range(0, Npigs):
            pig1 = SelPigs[p1]
            df1 = pig1.ptype.diplength/DipLengths[p1]
            for p2 in range(0, p1):
                pig2 = SelPigs[p2]
                df2 = pig2.ptype.diplength/DipLengths[p2]
                for atm in range(0, len(pig1.ptype.q10)):
                    for atn in range(0, len(pig2.ptype.q10)):
                        Rmn = CoordList[p1][atm,:] - CoordList[p2][atn,:]
                        rmn = np.linalg.norm(Rmn)
                        CoupMat[p1,p2] += df1*df2*(pig1.ptype.q10[atm]*pig2.ptype.q10[atn]/rmn)*(Erg2J/h)/c
        
        CoupMat = CoupMat + np.transpose(CoupMat)
        
    return CoupMat, Dips

def build_excitons(b):
    speclbl_out.clear_output()
    if coupselect.value=='TrESP':
        Coups, Dips = calculate_tresp()
        
    if siteselect.value=='Uniform':
        Freqs = []
        for p in range(0, len(PigList)):
            if ChainList.count(PigList[p].chain)>0:
                pname = PigList[p].ptype.stdname
                if pname=='CLA' or pname=='PHA':
                    freq = (1.0e+7)/670.0
                elif pname=='CLB' or pname=='PHB':
                    freqs = (1.0e+7)/655.0
                elif pname[0]=='B':
                    freq = (1.0e+7)/800.0
                else:
                    freq = (1.0e+7)/670.0
                Freqs.append(freq)
                
    # If we successfully calculated couplign constants
    if len(Dips)>0:
        comment_string = ''
        for p in range (0, len(PigList)):
            pig = PigList[p]
            if ChainList.count(pig.chain)>0:
                comment_string += pig.resname + ":" + pig.chain + str(pig.resnum) + ' '
        np.savetxt('misc/coups.txt', Coups, header = comment_string)
        np.savetxt('misc/freqs.txt', Freqs, header = comment_string)
        np.savetxt('misc/dips.txt', Dips, header = comment_string)
        with open("misc/names.txt", "w") as file:
            file.write(comment_string)
        display_markdown('Couplings saved to <a href=\"misc/coups.txt\" target="_blank">coups.txt</a>', raw=True)
        display_markdown('Frequencies saved to <a href=\"misc/freqs.txt\" target="_blank">freqs.txt</a>', raw=True)
        display_markdown('Dipoles saved to <a href=\"misc/dips.txt\" target="_blank">dips.txt</a>', raw=True)
        
        with speclbl_out:
            display_markdown('<a href=\"Spectrum.ipynb\" target="_blank">Load Spectrum Calculator</a>', raw=True)
    else:
        speclbl_out
        
        
couplbl = widgets.Label(value='Coupling Model')
coupselect = widgets.RadioButtons(
    options=['TrESP'],
    value='TrESP',
    disabled=True,
    layout=Layout(width='2cm')
)

sitelbl = widgets.Label(value='Site Energy Model')
siteselect = widgets.RadioButtons(
    options=['Uniform'],
    value='Uniform',
    disabled=True,
    layout=Layout(width='2cm')
)

excgo = widgets.Button(
    description='Go!',
    disabled=True,
    tooltip='Click to build an exciton model'
)

excgo.on_click(build_excitons)

speclbl_out = widgets.Output()


excbox = widgets.VBox(
    [
        widgets.HBox([
            widgets.VBox([couplbl, coupselect], layout=Layout(width='3cm', margin='5pt')),
            widgets.VBox([sitelbl, siteselect], layout=Layout(width='3cm', margin='5pt')),
        ], layout=Layout(width='7cm', margin='5pt'), ),
        excgo,
        speclbl_out,
    ], layout=Layout(align_items='center', width='7.5cm'))


##################################################################
# mdbox: Interface for preparing MD models
##################################################################
mdbox = widgets.VBox([])

#mainacc = widgets.Accordion(children=[strucbox, selbox, writebox, excbox, mdbox], layout=Layout(width='8cm'))
mainacc = widgets.Accordion(children=[strucbox, selbox, writebox, excbox], layout=Layout(width='8cm'))
mainacc.set_title(0, 'Load')
mainacc.set_title(1, 'Select')
mainacc.set_title(2, 'Write PDB')
mainacc.set_title(3, 'Prepare Exciton Model')
#mainacc.set_title(4, 'Prepare MD Topology')

pigbox = widgets.HBox()
pigacc = widgets.Accordion(children=[pigbox])
pigacc.set_title(0, 'Pigment List')

mainbox = widgets.VBox([widgets.HBox([pdbview, mainacc]), pigacc])
display(mainbox)

def update_pigtypes(change):
    global PigList
    
    for pig in PigList:
        if pig.widget.value!=pig.ptype.name:
            for typ in pig.alist:
                if typ.name==pig.widget.value:
                    pig.ptype = typ
                    break


def build_pigbox():
    global pigbox
    global PigList
    
    pigbox.layout = Layout(display='flex',
                    flex_flow='row wrap',
                    align_items='stretch',
                    width='100%')

    hlist = []
    for chain in ChainList:
        chwidglist = []
        for p in range(0, len(PigList)):
            pig = PigList[p]
            res = struc.residues[pig.idx]
            if res.chain==chain:
                dropbox = widgets.Dropdown(
                            options=[typ.name for typ in pig.alist],
                            value=pig.ptype.name,
                            description=res.name + ' ' + chain + str(res.number),
                            disabled=False,
                            layout=Layout(width='4.5cm')
                        )
                dropbox.observe(update_pigtypes, 'value')
                pig.widget = dropbox
                chwidglist.append(dropbox)
        if len(chwidglist)>0:
            lbl = widgets.Label(value='Chain ' + chain + ":")
            hlist.append(widgets.VBox([lbl] + chwidglist))
    pigbox.children = hlist


In [1]:
import ipywidgets as widgets
import os
from IPython.display import display
from IPython.display import display_markdown


def copy_exercise(self):
    uname = txt_uname.value.replace(" ", "_").lower()
    #fpath = "~/MOLSPEC/local/"
    fpath = "../../../../local/"
    fname = "exercise12_" + uname + ".ipynb"
    
    
    if len(uname)<=0:
        print('Please enter a valid user name!')
    elif os.path.isfile(fpath+fname) and cb_overwrite.value==False:
        print('The file already exists! To overwrite check the \"Overwrite Existing\" box and try again.')
        FancyText = "Click [here](" + fpath + fname + ") to open existing copy."
        display_markdown(FancyText, raw=True)
    else:
        out = !{"cp exercise12.ipynb " + fpath+fname}
        if len(out)>0:
            for line in out:
                print(out)
        else:
            FancyText = "Successfully copied exercise to local directory!<br> Click [here](" + fpath + fname + ") to open."
            display_markdown(FancyText, raw=True)
    
txt_uname = widgets.Text(
    value='',
    placeholder='User name',
    description='Purdue ID:',
    disabled=False
)


bt_genfile = widgets.Button(
    description='Copy Exercise',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Enter your username and then click to create a local exercise file'
)



cb_overwrite = widgets.Checkbox(
    value=False,
    description='Overwrite Existing?',
    disabled=False
)


bt_genfile.on_click(copy_exercise)

display(widgets.HBox([txt_uname, bt_genfile, cb_overwrite]))
